## 학습정리

### 21장. 자연어 처리
* 정성적인 특징
####  21.1 워드 클라우드
####  21.2 n-그램 언어모델
####  21.3 문법 규칙
####  21.4 여담 : 깁스 샘플링
####  21.5 토픽 모델링
####  21.6 단어 벡터
####  21.7 재귀 신경망
####  21.8 예시 : 문자 단위의 RNN 사용하기
####  21.9 더 공부해 보고 싶다면
